In [2]:
import pandas as pd
import gffutils
import os
import pybedtools
from pybedtools.featurefuncs import TSS
from gffutils.helpers import asinterval

def tss_generator():
    for transcript in db.features_of_type('transcript'):
        yield TSS(asinterval(transcript),upstream=1,downstream=0)

samplefile = "/ifs3/scdata/RNA-Seq_counts/sample"
samplefilepath = pd.read_csv(samplefile,header=None)

In [8]:
for i in range(len(samplefilepath)):
    samplefilepath[0][i]
    input = samplefilepath[0][i]+"/transcript_anno_duplicates.bed"
    db_file = samplefilepath[0][i]+'/transcript_anno_duplicates.db'
    outdir = samplefilepath[0][i]
    
    
    # Check if the database already exists, if not create it
    if not os.path.exists(db_file):
        gffutils.create_db(input, dbfn=db_file, force=True, keep_order=True, merge_strategy='merge', disable_infer_genes=True,disable_infer_transcripts=True,sort_attribute_values=True)

    db = gffutils.FeatureDB(db_file)

    tsses = pybedtools.BedTool(tss_generator()).saveas(samplefilepath[0][i]+'/transcript_tsses.gtf')

    tsses_10_4kb = tsses.slop(l=10000,r=4000,genome='hg38',output=samplefilepath[0][i]+'/transcript_tsses_-10-4kb.gtf')
    tsses_10_4kb = tsses.slop(l=10000,r=4000,genome='hg38',output=samplefilepath[0][i]+'/transcript_tsses_-10-4kb.bed')
